In [1]:
# we scraper
from bs4 import BeautifulSoup

# request
import requests

# io
from io import BytesIO

# date parser and sleep
from dateutil.parser import parse
import time

# web driver
from datetime import datetime
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc

# reeading and exporting to excel files
import pandas as pd

# looping process monitoring
from tqdm import tqdm
import random

In [2]:
# class to determine ganjil genap
def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False

s = slice(11)

In [51]:
# reading all BL number from google sheet
from acquiringbl import takingBL
bl_list = takingBL("SEALAND")

In [52]:
len(bl_list)

32

In [53]:
dict_hasil = {}
current_dict = {}
list_dict = []
gagal = []

In [38]:
# bl_list = ['228576712', '228329054', '231591184']

**FIX**


In [54]:
list_dict = []
gagal = []
# web scripting
options = Options()
options.add_argument("--window-size=1920,1280")
driver = uc.Chrome(use_subprocess=True)
driver.get('https://www.sealandmaersk.com/tracking/' + bl_list[random.randint(0, len(bl_list)-1)])

# wait to load all
time.sleep(2)

#click allow all
click_allowall = driver.find_element(By.XPATH,'//button[text()="Allow all"]')
click_allowall.click()

# click asia
# click_asia = driver.find_element(By.XPATH,"/html/body/div[2]/div/section/div/div[2]/form/div[1]/div/div/ol/li[3]/label/div[1]")
click_asia = driver.find_element(By.XPATH,"/html/body/div[3]/div/section/div/div[2]/form/div[1]/div/div/ol/li[3]/label/div[1]")

click_asia.click()

# click continue
click_continue = driver.find_element(By.ID,'confirmRegion')
click_continue.click()

for i, bls in enumerate(tqdm(bl_list)):
    try:
        # access the web
        driver.get('https://www.sealandmaersk.com/tracking/' + bl_list[i])

        # sleep wait
        # title = driver.find_element(By.XPATH, "/html/body/main/div/div/dl")
        # waiting for page tittle to load
        myElem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "/html/body/main/div/div/dl")))
        time.sleep(3)
        
        # finding how many hide details in 1 page refering to how many container number in 1 bl
        soup1 = BeautifulSoup(driver.page_source, 'lxml')
        containers = soup1.find_all('div', {"data-test": "container"})
        number_of_ctr = len(containers)
        
        print(bls, "HAVE ", number_of_ctr, " CTR")
        # print(dict_hasil)

        # print("current bl: ", bls, "\n number of container: ", number_of_ctr)
        if number_of_ctr > 1:
            # taking BL Number, From, and To 
            title = driver.find_element(By.XPATH, "/html/body/main/div/div/dl")
            titles = title.text.split('\n')
            titles.remove('View Shipment Details')
            print(titles)
            for a, item in enumerate(titles):
                #
                if a%2==0 :
                    case_judul = {titles[a]:titles[a+1]}
                    dict_hasil.update(case_judul)
            
            print("Hasil dari Append Judul")
            print(dict_hasil)
            if driver.find_element(By.CLASS_NAME, "track__feedback"):
                x=4
            else:
                x=3
            # clicking all view more
            for c in range(number_of_ctr):
                try:
                    print("clicking div no :",  x)
                    show_details2 = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/div/div/div[{}]/dl/button'.format(x))))  
                    show_details_check = driver.find_element(By.XPATH, "/html/body/main/div/div/div[{}]/dl/button/span".format(x))
                    if show_details_check.text == 'Show details':
                        show_details2.click()
                    print(show_details_check.text)
                    
                    time.sleep(1)
                except Exception as e:
                    print(e)
                x=x+1

            time.sleep(2)
            # taking the whole page's html
            soup2 = BeautifulSoup(driver.page_source, 'lxml')
            
            table = soup2.find_all('div', {"data-test": "container"})
            milestones = []
            for tag in table:
                for ctri1, ctr_num1  in enumerate(tag.find_all('strong', {"data-test":'container-details-value'})):
                    temp_ctr_num1 = ctr_num1
                    milestones.append("Container Number")
                    milestones.append(temp_ctr_num1.text)
                for m, li_milestone in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-complete'})):
                    span_history = li_milestone.find_all("span")
                    for c1, city1 in enumerate(li_milestone.find_all("strong")):
                        temp1 = city1.text
                    for milestone_span in span_history:
                        if is_date(milestone_span.text):
                            milestones.append(milestone_span.text[s])
                        else:
                            milestones.append(temp1 +" "+ milestone_span.text)
                            
                for n, li_expected in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-future'})):
                    spans = li_expected.find_all("span")
                    for c2, city2 in enumerate(li_expected.find_all("strong")):
                        temp2 = city2.text
                    for expected_span in spans:
                        if is_date(expected_span.text):
                            milestones.append(expected_span.text[s])
                        else:
                            milestones.append(temp2 + " ESTIMATED " + expected_span.text)
                 
                for panjang in range(0, len(milestones), 2):
                    key = milestones[panjang]
                    value = milestones[panjang + 1]

                    if key == 'Container Number':
                        if current_dict:
                            list_dict.append(current_dict.copy())
                        current_dict = {'Container Number': value}
                    else:
                        current_dict[key] = value

                    current_dict.update(dict_hasil) 
                print(current_dict)
            list_dict.append(current_dict)
            current_dict = {}
            dict_hasil={}
            print(bls, ", completed!!")
        else:
            time.sleep(1)
            
            title = driver.find_element(By.XPATH, "/html/body/main/div/div/dl")
            titles = title.text.split('\n')
            titles.remove('View Shipment Details')
            for a, item in enumerate(titles):
                if a%2==0 :
                    case_judul = {titles[a]:titles[a+1]}
                    dict_hasil.update(case_judul)
            
            time.sleep(2)
            soup2 = BeautifulSoup(driver.page_source, 'lxml')
            
            table = soup2.find_all('div', {"data-test": "container"})
            milestones = []
            for tag in table:
                for ctri1, ctr_num1  in enumerate(tag.find_all('strong', {"data-test":'container-details-value'})):
                    temp_ctr_num1 = ctr_num1
                    milestones.append("Container Number")
                    milestones.append(temp_ctr_num1.text)

                for m, li_milestone in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-complete'})):
                    span_history = li_milestone.find_all("span")
                    for c1, city1 in enumerate(li_milestone.find_all("strong")):
                        temp1 = city1.text
                    for milestone_span in span_history:
                        if is_date(milestone_span.text):
                            milestones.append(milestone_span.text[s])
                        else:
                            milestones.append(temp1 +" "+ milestone_span.text)
                            
                for n, li_expected in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-future'})):
                    spans = li_expected.find_all("span")
                    for c2, city2 in enumerate(li_expected.find_all("strong")):
                        temp2 = city2.text
                    for expected_span in spans:
                        if is_date(expected_span.text):
                            milestones.append(expected_span.text[s])
                        else:
                            milestones.append(temp2 + " ESTIMATED " + expected_span.text)
            
            for t, milestone in enumerate(milestones):
                if milestone == "Container Number":
                    case_mlstn = {milestones[t]:milestones[t+1]}
                    current_dict.update(case_mlstn)
                elif t%2==0 :
                    case_mlstn = {milestones[t]:milestones[t+1]}
                    current_dict.update(case_mlstn)
                    
            current_dict.update(dict_hasil) 
            print(current_dict)
            list_dict.append(current_dict)
            current_dict = {}
            dict_hasil={}
        
        unique_list = []
        for d in list_dict:
            if d not in unique_list:
                unique_list.append(d)

    except Exception as e:
        print(e)
        print("{} GAGAL!!".format(bls))
        gagal.append(bls)

  0%|          | 0/32 [00:00<?, ?it/s]

228151079 HAVE  1  CTR


  3%|▎         | 1/32 [00:10<05:18, 10.26s/it]

{'Container Number': 'GCXU5783600', 'Jakarta Gate outEmpty': '08 Jun 2023', 'Jakarta Gate in': '09 Jun 2023', 'Jakarta Load': '16 Jun 2023', 'Jakarta Vessel departure': '16 Jun 2023', 'Subic Vessel arrival': '27 Jun 2023', 'Subic Discharge': '28 Jun 2023', 'Bill of Lading number': '228151079', 'From': 'Jakarta', 'To': 'Subic'}
228627103 HAVE  1  CTR


  6%|▋         | 2/32 [00:20<05:03, 10.10s/it]

...

{'Container Number': 'TLLU5971854', 'Jakarta Gate outEmpty': '28 Jun 2023', 'Jakarta Gate in': '01 Jul 2023', 'Jakarta Load': '07 Jul 2023', 'Jakarta Vessel departure': '07 Jul 2023', 'Subic Vessel arrival': '19 Jul 2023', 'Subic Discharge': '20 Jul 2023', 'Bill of Lading number': '228776329', 'From': 'Jakarta', 'To': 'Subic'}
231590994 HAVE  1  CTR


 97%|█████████▋| 31/32 [05:51<00:10, 10.42s/it]

{'Container Number': 'TRHU8031970', 'Jakarta Gate outEmpty': '23 Sep 2023', 'Jakarta Gate in': '25 Sep 2023', 'Jakarta Load': '29 Sep 2023', 'Jakarta Vessel departure': '29 Sep 2023', 'Tanjung Pelepas Vessel arrival': '01 Oct 2023', 'Tanjung Pelepas Discharge': '01 Oct 2023', 'Tanjung Pelepas Load': '05 Oct 2023', 'Tanjung Pelepas Vessel departure': '06 Oct 2023', 'Bintulu Vessel arrival': '10 Oct 2023', 'Bintulu Discharge': '12 Oct 2023', 'Bintulu Gate out': '13 Oct 2023', 'Miri Container drop off at customer site': 'Miri ', 'Bintulu Empty container return': '16 Oct 2023', 'Bill of Lading number': '231590994', 'From': 'Jakarta', 'To': 'Miri'}
228495628 HAVE  3  CTR
['Bill of Lading number', '228495628', 'From', 'Jakarta', 'To', 'Subic']
Hasil dari Append Judul
{'Bill of Lading number': '228495628', 'From': 'Jakarta', 'To': 'Subic'}
clicking div no : 4
Hide details
clicking div no : 5
Hide details
clicking div no : 6
Hide details


100%|██████████| 32/32 [06:04<00:00, 11.41s/it]

{'Container Number': 'MRSU3497495', 'Bill of Lading number': '228495628', 'From': 'Jakarta', 'To': 'Subic', 'Jakarta Gate outEmpty': '21 Jun 2023', 'Jakarta Gate in': '24 Jun 2023', 'Jakarta Load': '30 Jun 2023', 'Jakarta Vessel departure': '30 Jun 2023', 'Subic Vessel arrival': '12 Jul 2023', 'Subic Discharge': '12 Jul 2023'}
{'Container Number': 'TRHU5089617', 'Bill of Lading number': '228495628', 'From': 'Jakarta', 'To': 'Subic', 'Jakarta Gate outEmpty': '22 Jun 2023', 'Jakarta Gate in': '24 Jun 2023', 'Jakarta Load': '30 Jun 2023', 'Jakarta Vessel departure': '30 Jun 2023', 'Subic Vessel arrival': '12 Jul 2023', 'Subic Discharge': '12 Jul 2023'}
{'Container Number': 'TGBU8899367', 'Bill of Lading number': '228495628', 'From': 'Jakarta', 'To': 'Subic', 'Jakarta Gate outEmpty': '21 Jun 2023', 'Jakarta Gate in': '24 Jun 2023', 'Jakarta Load': '30 Jun 2023', 'Jakarta Vessel departure': '30 Jun 2023', 'Subic Vessel arrival': '12 Jul 2023', 'Subic Discharge': '12 Jul 2023'}
228495628 , c

In [55]:
unique_list

[{'Container Number': 'GCXU5783600',
  'Jakarta Gate outEmpty': '08 Jun 2023',
  'Jakarta Gate in': '09 Jun 2023',
  'Jakarta Load': '16 Jun 2023',
  'Jakarta Vessel departure': '16 Jun 2023',
  'Subic Vessel arrival': '27 Jun 2023',
  'Subic Discharge': '28 Jun 2023',
  'Bill of Lading number': '228151079',
  'From': 'Jakarta',
  'To': 'Subic'},
 {'Container Number': 'MRSU4977539',
  'Jakarta Gate outEmpty': '24 Jun 2023',
  'Jakarta Gate in': '26 Jun 2023',
  'Jakarta Load': '30 Jun 2023',
  'Jakarta Vessel departure': '30 Jun 2023',
  'Batangas Vessel arrival': '08 Jul 2023',
  'Batangas Discharge': '08 Jul 2023',
  'Bill of Lading number': '228627103',
  'From': 'Jakarta',
  'To': 'Batangas'},
 {'Container Number': 'CAAU6577730',
  'Jakarta Gate outEmpty': '01 Jun 2023',
  'Jakarta Gate in': '03 Jun 2023',
  'Jakarta Load': '09 Jun 2023',
  'Jakarta Vessel departure': '09 Jun 2023',
  'Subic Vessel arrival': '21 Jun 2023',
  'Subic Discharge': '22 Jun 2023',
  'Bill of Lading numbe

In [56]:
filtered_dict_list =[]
for dict_milestone in unique_list:
    # print(dict_milestone)
    kw = ['Bill of Lading number', 'Container Number', 'From', 'To', dict_milestone['From'], dict_milestone['To']]
    dict_key = []
    for keys in dict_milestone:
        dict_key.append(keys)
    # print("BEFORE", dict_key)
    new_list = [] 
    
    for key in dict_key:
        if any(substring in key for substring in kw):
            new_list.append(key)
    # print("AFTER",new_list)
    dict_milestone = {key: dict_milestone[key] for key in new_list}
    filtered_dict_list.append(dict_milestone)

In [57]:
filtered_dict_list

[{'Container Number': 'GCXU5783600',
  'Jakarta Gate outEmpty': '08 Jun 2023',
  'Jakarta Gate in': '09 Jun 2023',
  'Jakarta Load': '16 Jun 2023',
  'Jakarta Vessel departure': '16 Jun 2023',
  'Subic Vessel arrival': '27 Jun 2023',
  'Subic Discharge': '28 Jun 2023',
  'Bill of Lading number': '228151079',
  'From': 'Jakarta',
  'To': 'Subic'},
 {'Container Number': 'MRSU4977539',
  'Jakarta Gate outEmpty': '24 Jun 2023',
  'Jakarta Gate in': '26 Jun 2023',
  'Jakarta Load': '30 Jun 2023',
  'Jakarta Vessel departure': '30 Jun 2023',
  'Batangas Vessel arrival': '08 Jul 2023',
  'Batangas Discharge': '08 Jul 2023',
  'Bill of Lading number': '228627103',
  'From': 'Jakarta',
  'To': 'Batangas'},
 {'Container Number': 'CAAU6577730',
  'Jakarta Gate outEmpty': '01 Jun 2023',
  'Jakarta Gate in': '03 Jun 2023',
  'Jakarta Load': '09 Jun 2023',
  'Jakarta Vessel departure': '09 Jun 2023',
  'Subic Vessel arrival': '21 Jun 2023',
  'Subic Discharge': '22 Jun 2023',
  'Bill of Lading numbe

In [58]:
list_of_dict_fix = []
for filter_dict in filtered_dict_list:
    replacement_mapping = {
        filter_dict["From"]: 'Origin',
        filter_dict["To"]: 'Destination'
    }

    updated_dict = {}

    for key, value in filter_dict.items():
        for old_key, new_key in replacement_mapping.items():
            key = key.replace(old_key, new_key)
        updated_dict[key] = value

    list_of_dict_fix.append(updated_dict)

In [59]:
list_of_dict_fix

[{'Container Number': 'GCXU5783600',
  'Origin Gate outEmpty': '08 Jun 2023',
  'Origin Gate in': '09 Jun 2023',
  'Origin Load': '16 Jun 2023',
  'Origin Vessel departure': '16 Jun 2023',
  'Destination Vessel arrival': '27 Jun 2023',
  'Destination Discharge': '28 Jun 2023',
  'Bill of Lading number': '228151079',
  'From': 'Jakarta',
  'To': 'Subic'},
 {'Container Number': 'MRSU4977539',
  'Origin Gate outEmpty': '24 Jun 2023',
  'Origin Gate in': '26 Jun 2023',
  'Origin Load': '30 Jun 2023',
  'Origin Vessel departure': '30 Jun 2023',
  'Destination Vessel arrival': '08 Jul 2023',
  'Destination Discharge': '08 Jul 2023',
  'Bill of Lading number': '228627103',
  'From': 'Jakarta',
  'To': 'Batangas'},
 {'Container Number': 'CAAU6577730',
  'Origin Gate outEmpty': '01 Jun 2023',
  'Origin Gate in': '03 Jun 2023',
  'Origin Load': '09 Jun 2023',
  'Origin Vessel departure': '09 Jun 2023',
  'Destination Vessel arrival': '21 Jun 2023',
  'Destination Discharge': '22 Jun 2023',
  'Bi

In [60]:
list_of_dict_fix2 = []
for filter_dict in list_of_dict_fix:
    replacement_mapping = {
        "Bill of Lading number" : "BL Number",
        "Origin Vessel departure": 'ATD',
        "Destination Discharge": 'ATA',
        "Destination Gate out": 'Container Release',
        "Destination Empty container return" : 'Container Return'
    }

    updated_dict = {}

    for key, value in filter_dict.items():
        for old_key, new_key in replacement_mapping.items():
            key = key.replace(old_key, new_key)
        updated_dict[key] = value

        if is_date(value):
           input_date = datetime.strptime(value, '%d %b %Y')
           updated_dict[key] = input_date.strftime("%Y-%m-%d")
    
    updated_dict["Liners"] = "SEALAND"

    list_of_dict_fix2.append(updated_dict)

In [61]:
list_of_dict_fix2

[{'Container Number': 'GCXU5783600',
  'Origin Gate outEmpty': '2023-06-08',
  'Origin Gate in': '2023-06-09',
  'Origin Load': '2023-06-16',
  'ATD': '2023-06-16',
  'Destination Vessel arrival': '2023-06-27',
  'ATA': '2023-06-28',
  'BL Number': '228151079',
  'From': 'Jakarta',
  'To': 'Subic',
  'Liners': 'SEALAND'},
 {'Container Number': 'MRSU4977539',
  'Origin Gate outEmpty': '2023-06-24',
  'Origin Gate in': '2023-06-26',
  'Origin Load': '2023-06-30',
  'ATD': '2023-06-30',
  'Destination Vessel arrival': '2023-07-08',
  'ATA': '2023-07-08',
  'BL Number': '228627103',
  'From': 'Jakarta',
  'To': 'Batangas',
  'Liners': 'SEALAND'},
 {'Container Number': 'CAAU6577730',
  'Origin Gate outEmpty': '2023-06-01',
  'Origin Gate in': '2023-06-03',
  'Origin Load': '2023-06-09',
  'ATD': '2023-06-09',
  'Destination Vessel arrival': '2023-06-21',
  'ATA': '2023-06-22',
  'BL Number': '228000484',
  'From': 'Jakarta',
  'To': 'Subic',
  'Liners': 'SEALAND'},
 {'Container Number': 'CA

In [50]:
# connect to mongodb
from mongoinit import mongo_table_initiation, insert_many_mongo

mongo_table_initiation()
insert_many_mongo(list_of_dict_fix2)

Today's Collection Name ===>  all_tracking_Oct-16-2023
Today's Collection Has Been Made
Inserting Many Complete!!


In [62]:
gagal

['228883601']

In [63]:
df = pd.DataFrame.from_dict(list_of_dict_fix2)
df.to_excel('sealand_maintained.xlsx')

In [16]:
list_dict = []
# web scripting
options = Options()
options.add_argument("--window-size=1920,1280")
driver = uc.Chrome(use_subprocess=True)
driver.get('https://www.sealandmaersk.com/tracking/' + bl_list[random.randint(0, len(bl_list)-1)])

# wait to load all
time.sleep(2)

#click allow all
click_allowall = driver.find_element(By.XPATH,'//button[text()="Allow all"]')
click_allowall.click()

# click asia
click_asia = driver.find_element(By.XPATH,"/html/body/div[2]/div/section/div/div[2]/form/div[1]/div/div/ol/li[3]/label/div[1]")
click_asia.click()

# click continue
click_continue = driver.find_element(By.ID,'confirmRegion')
click_continue.click()

for i, bls in enumerate(tqdm(gagal)):
    try:
        # access the web
        driver.get('https://www.sealandmaersk.com/tracking/' + bl_list[i])

        # sleep wait
        time.sleep(3)
        
        # finding how many hide details in 1 page refering to how many container number in 1 bl
        soup1 = BeautifulSoup(driver.page_source, 'lxml')
        containers = soup1.find_all('div', {"data-test": "container"})
        number_of_ctr = len(containers)
        
        print(bls, "HAVE ", number_of_ctr, " CTR")
        # print(dict_hasil)

        # print("current bl: ", bls, "\n number of container: ", number_of_ctr)
        if number_of_ctr > 1:
            # taking BL Number, From, and To 
            title = driver.find_element(By.XPATH, "/html/body/main/div/div/dl")
            titles = title.text.split('\n')
            titles.remove('View Shipment Details')
            print(titles)
            for a, item in enumerate(titles):
                #
                if a%2==0 :
                    case_judul = {titles[a]:titles[a+1]}
                    dict_hasil.update(case_judul)
            
            print("Hasil dari Append Judul")
            print(dict_hasil)
            x=3
            # clicking all view more
            for c in range(number_of_ctr):
                try:
                    print("clicking div no :",  x)
                    show_details2 = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/div/div/div[{}]/dl/button'.format(x))))  
                    show_details2.click()
                    time.sleep(1)
                except Exception as e:
                    print(e)
                x=x+1

            time.sleep(2)
            # taking the whole page's html
            soup2 = BeautifulSoup(driver.page_source, 'lxml')
            
            table = soup2.find_all('div', {"data-test": "container"})
            milestones = []
            for tag in table:
                for ctri1, ctr_num1  in enumerate(tag.find_all('strong', {"data-test":'container-details-value'})):
                    temp_ctr_num1 = ctr_num1
                    milestones.append("Container Number")
                    milestones.append(temp_ctr_num1.text)
                for m, li_milestone in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-complete'})):
                    span_history = li_milestone.find_all("span")
                    for c1, city1 in enumerate(li_milestone.find_all("strong")):
                        temp1 = city1.text
                    for milestone_span in span_history:
                        if is_date(milestone_span.text):
                            milestones.append(milestone_span.text[s])
                        else:
                            milestones.append(temp1 +" "+ milestone_span.text)
                            
                for n, li_expected in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-future'})):
                    spans = li_expected.find_all("span")
                    for c2, city2 in enumerate(li_expected.find_all("strong")):
                        temp2 = city2.text
                    for expected_span in spans:
                        if is_date(expected_span.text):
                            milestones.append(expected_span.text[s])
                        else:
                            milestones.append(temp2 + " ESTIMATED " + expected_span.text)
                 
                for panjang in range(0, len(milestones), 2):
                    key = milestones[panjang]
                    value = milestones[panjang + 1]

                    if key == 'Container Number':
                        if current_dict:
                            list_dict.append(current_dict.copy())
                        current_dict = {'Container Number': value}
                    else:
                        current_dict[key] = value

                    current_dict.update(dict_hasil) 
                print(current_dict)
            list_dict.append(current_dict)
            current_dict = {}
            dict_hasil={}
            print(bls, ", completed!!")
        else:
            time.sleep(1)
            
            title = driver.find_element(By.XPATH, "/html/body/main/div/div/dl")
            titles = title.text.split('\n')
            titles.remove('View Shipment Details')
            for a, item in enumerate(titles):
                if a%2==0 :
                    case_judul = {titles[a]:titles[a+1]}
                    dict_hasil.update(case_judul)
            
            time.sleep(2)
            soup2 = BeautifulSoup(driver.page_source, 'lxml')
            
            table = soup2.find_all('div', {"data-test": "container"})
            milestones = []
            for tag in table:
                for ctri1, ctr_num1  in enumerate(tag.find_all('strong', {"data-test":'container-details-value'})):
                    temp_ctr_num1 = ctr_num1
                    milestones.append("Container Number")
                    milestones.append(temp_ctr_num1.text)

                for m, li_milestone in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-complete'})):
                    span_history = li_milestone.find_all("span")
                    for c1, city1 in enumerate(li_milestone.find_all("strong")):
                        temp1 = city1.text
                    for milestone_span in span_history:
                        if is_date(milestone_span.text):
                            milestones.append(milestone_span.text[s])
                        else:
                            milestones.append(temp1 +" "+ milestone_span.text)
                            
                for n, li_expected in enumerate(tag.find_all('li', {"data-test":'transport-plan-item-future'})):
                    spans = li_expected.find_all("span")
                    for c2, city2 in enumerate(li_expected.find_all("strong")):
                        temp2 = city2.text
                    for expected_span in spans:
                        if is_date(expected_span.text):
                            milestones.append(expected_span.text[s])
                        else:
                            milestones.append(temp2 + " ESTIMATED " + expected_span.text)
            
            for t, milestone in enumerate(milestones):
                if milestone == "Container Number":
                    case_mlstn = {milestones[t]:milestones[t+1]}
                    current_dict.update(case_mlstn)
                elif t%2==0 :
                    case_mlstn = {milestones[t]:milestones[t+1]}
                    current_dict.update(case_mlstn)
                    
            current_dict.update(dict_hasil) 
            list_dict.append(current_dict)
            current_dict = {}
            dict_hasil={}
        
        unique_list = []
        for d in list_dict:
            if d not in unique_list:
                unique_list.append(d)

    except Exception as e:
        print(e)
        print("{} GAGAL!!".format(bls))
        gagal.append(bls)

  0%|          | 0/1 [00:00<?, ?it/s]

231590994 HAVE  1  CTR


100%|██████████| 1/1 [00:08<00:00,  8.01s/it]


In [17]:
unique_list

[{'Container Number': 'CAAU6577730',
  'Jakarta Gate outEmpty': '01 Jun 2023',
  'Jakarta Gate in': '03 Jun 2023',
  'Jakarta Load': '09 Jun 2023',
  'Jakarta Vessel departure': '09 Jun 2023',
  'Subic Vessel arrival': '21 Jun 2023',
  'Subic Discharge': '22 Jun 2023',
  'Bill of Lading number': '228000484',
  'From': 'Jakarta',
  'To': 'Subic'}]

In [13]:
from datetime import datetime
#declarate get

date = datetime.today().strftime("%d.%m.%Y")
print(date)

11.10.2023
